In [1]:
import pandas as pd 
import numpy as np 
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout,BatchNormalization,Conv1D,GlobalAveragePooling1D,Activation, Embedding

training_set=pd.read_csv('../dataset/labeledTrainData.tsv',sep='\t')
training_set.head(2)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."


In [2]:
# train_test_split
y_train=training_set['sentiment'].values
x_train=training_set['review'].values
train_data,test_data,train_labels,test_labels=train_test_split(x_train,y_train,shuffle=True,test_size=0.2,random_state=42,stratify=y_train)

In [3]:
y_train = [[1,0] if y==1 else [0,1] for y in train_labels ]
y_test = [[1,0] if y==1 else [0,1] for y in test_labels ]

y_train = np.array(y_train)
y_test = np.array(y_test)

In [4]:
max_features = 8192
maxlen = 128
embed_size = 64

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data)

# save tokenizer for testing
# pickle.dump(tokenizer,open('tokenizer.sav','wb'),protocol=pickle.HIGHEST_PROTOCOL)

token_train = tokenizer.texts_to_sequences(train_data)
token_test = tokenizer.texts_to_sequences(test_data)


In [6]:
x_train = pad_sequences(token_train, maxlen=maxlen)
x_test = pad_sequences(token_test, maxlen=maxlen)

In [20]:
input = Input(shape=(maxlen,))
net = Embedding(max_features, embed_size)(input)
net = Dropout(0.2)(net)

# net = BatchNormalization()(net)
# net = Conv1D(32, 7, padding='same', activation='relu')(net)

net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)

net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)

net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)

net1 = BatchNormalization()(net)
net = Conv1D(2, 1)(net)
net = GlobalAveragePooling1D()(net)
output = Activation('softmax')(net)
model = Model(inputs = input, outputs = output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [8]:
# model.summary()

In [21]:
model.fit(x_train, y_train, batch_size=1024, epochs=6, validation_split=0.1)

Epoch 1/6


ValueError: in user code:

    C:\Users\rabin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\rabin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\rabin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\rabin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\rabin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\rabin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

In [13]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 1s 7ms/step - loss: 0.6942 - acc: 0.5000


[0.6941590309143066, 0.5]

In [19]:
test_input = tokenizer.texts_to_sequences(['good','bad','intresting'])
print(test_input)

test_input = pad_sequences(test_input, maxlen=maxlen)
# print(test_input)

model.predict(test_input)

[[49], [75], []]


array([[0.61791   , 0.38208994],
       [0.6168882 , 0.38311177],
       [0.617769  , 0.382231  ]], dtype=float32)